<a href="https://colab.research.google.com/github/MengOonLee/BertelsmannAITrack/blob/master/NeuralNetwork/MiniFlow/Fundamental.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%file ./miniflow.py

class Node(object):
  def __init__(self, inbound_nodes=[]):
    # Node(s) from which this Node receives values
    self.inbound_nodes = inbound_nodes
    # Node(s) to which this Node passes values
    self.outbound_nodes = []
    # For each inbound_node, add the current Node as outbound_node on its inputs.
    for n in self.inbound_nodes:
      n.outbound_nodes.append(self)
    # A calculated value
    self.value = None

  # These will be implemented in a subclass.
  def forward(self):
    """
    Forward propagation.
    
    Compute the output value based on `inbound_nodes` 
    and store the result in self.value.
    """
    raise NotImplemented

class Input(Node):
  def __init__(self):
    # An Input node has no inbound nodes,
    # so no need to pass anything to the Node instantiator
    Node.__init__(self)

  # Note: Input node is the only node where the value
  # may be passed as an argument to forward().
  #
  # All other node implementations should calculate their
  # values from the value of previous node, using self.inbound_nodes
  #
  # Example:
  # val0 = self.inbound_nodes[0].value
  def forward(self, value=None):
    # Overwrite the value if one is passed in.
    if value is not None:
      self.value = value

class Linear(Node):
  def __init__(self, inputs, weights, bias):
    Node.__init__(self, [inputs, weights, bias]) # calls Node's constructor

    # NOTE: The weights and bias properties here are not
    # numbers, but rather references to other nodes.
    # The weight and bias values are stored within the 
    # respective nodes.

  def forward(self):
    """
    You could access the value in forward with self.inbound_nodes[i].value
    Set self.value to the value of the linear function output.
    """
    inputs = self.inbound_nodes[0].value
    weights = self.inbound_nodes[1].value
    bias = self.inbound_nodes[2].value
    self.value = bias
    for x, w in zip(inputs, weights):
      self.value += x*w

def topological_sort(feed_dict):
  """
  Sort generic nodes in topological order using Kahn's Algorithm.
  `feed_dict`: A dictionary where the key is a `Input` node 
  and the value is the respective value feed to that node.
  Returns a list of sorted nodes.
  """

  input_nodes = [n for n in feed_dict.keys()]

  G = {}
  nodes = [n for n in input_nodes]
  while len(nodes) > 0:
    n = nodes.pop(0)
    if n not in G:
      G[n] = {'in': set(), 'out': set()}
    for m in n.outbound_nodes:
      if m not in G:
        G[m] = {'in': set(), 'out': set()}
      G[n]['out'].add(m)
      G[m]['in'].add(n)
      nodes.append(m)

  L = []
  S = set(input_nodes)
  while len(S) > 0:
    n = S.pop()

    if isinstance(n, Input):
        n.value = feed_dict[n]

    L.append(n)
    for m in n.outbound_nodes:
        G[n]['out'].remove(m)
        G[m]['in'].remove(n)
        # if no other incoming edges add to S
        if len(G[m]['in'])==0:
            S.add(m)
  return L

def forward_pass(output_node, sorted_nodes):
  """
  Performs a forward pass through a list of sorted nodes.
  Arguments:
    `output_node`: A node in the graph, should be the output node 
      (have no outgoing edges).
    `sorted_nodes`: A topologically sorted list of nodes.
  Returns the output Node's value
  """

  for n in sorted_nodes:
    n.forward()

  return output_node.value

Writing ./miniflow.py


## This script builds and runs a graph with miniflow.

In [2]:
"""
This script builds and runs a graph with miniflow.
NOTE: Here we're using an Input node for more than a scalar.
In the case of weights and inputs the value of the Input node is
actually a python list!

In general, there's no restriction on the values that 
can be passed to an Input node.
"""

from miniflow import *

# Define `Input` nodes.
inputs, weights, bias = Input(), Input(), Input()

# Define and `Linear` node, the above `Input` nodes being the input.
f = Linear(inputs, weights, bias)

# The value of `Input` nodes will be set to values respectively.
feed_dict = {
  inputs: [6,14,3], 
  weights: [0.5, 0.25, 1.4],
  bias: 2
}

# Sort the nodes with topological sort.
graph = topological_sort(feed_dict)

output = forward_pass(f, graph)

# NOTE: because topological_sort sets the values for the `Input` nodes we could also access
# the value for x with x.value.
print("output = {} ".format(output))

output = 12.7 
